In [1]:
# Import libraries
import pandas as pd
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt

# Download historical data
data = yf.download('BTC-USD', start='2023-01-01')

# Calculate the 10-day high
data['10_day_high'] = data['Close'].rolling(window=10).max()

# Create a signal when the close price crosses the 10_day_high
data['buy_signal'] = np.where(data['Close'] > data['10_day_high'].shift(), 1, 0)

# We sell one day after buying
data['sell_signal'] = data['buy_signal'].shift()

# Set the sell signal column to zero initially
data['sell_signal'].fillna(0, inplace=True)

# Calculate daily strategy returns
data['strategy_returns'] = np.where(data['sell_signal'] == 1, data['Close'].pct_change(), 0)

# Calculate cumulative strategy returns
data['strategy_cumulative_returns'] = (1 + data['strategy_returns']).cumprod()

# Plot the equity curve of the strategy and benchmark
plt.figure(figsize=(10,5))
plt.plot(data['strategy_cumulative_returns'], label='10 Day High Strategy')
plt.legend()
plt.show()


ImportError: cannot import name 'axes' from 'matplotlib' (C:\Users\mehdi\AppData\Local\Programs\Python\Python310\lib\site-packages\matplotlib\__init__.py)

In [2]:
# Import libraries
import pandas as pd
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt

# Download historical data
data = yf.download('BTC-USD', start='2023-01-01')

# Calculate the 10-day high
data['10_day_high'] = data['Close'].rolling(window=5).max()

# Create a signal when the close price crosses the 10_day_high
data['buy_signal'] = np.where(data['Close'] > data['10_day_high'].shift(), 1, 0)

# Create a column that contains the price at which we last bought
data['buy_price'] = np.where(data['buy_signal'], data['Close'], np.nan)
data['buy_price'].ffill(inplace=True)

# Create a signal when the high price reaches 1% above the buy price
data['take_profit_signal'] = np.where(data['High'] > data['buy_price'] * 1.01, 1, 0)

# We sell one day after buying or when take profit signal is activated
data['sell_signal'] = np.where((data['buy_signal'].shift() == 1) | (data['take_profit_signal'] == 1), 1, 0)

# Set the sell signal column to zero initially
data['sell_signal'].fillna(0, inplace=True)

# Calculate daily strategy returns
data['strategy_returns'] = np.where(data['sell_signal'] == 1, np.where(data['take_profit_signal'] == 1, 0.01, (data['Close'] / data['buy_price'].shift() - 1)), 0)

# Calculate cumulative strategy returns
data['strategy_cumulative_returns'] = (1 + data['strategy_returns']).cumprod()

# Plot the equity curve of the strategy
plt.figure(figsize=(10,5))
plt.plot(data['strategy_cumulative_returns'], label='10 Day High Strategy with 1% Take Profit')
plt.legend()
plt.show()

# Calculate CAGR
years = (data.index[-1] - data.index[0]).days / 365.25
cagr = (data['strategy_cumulative_returns'][-1]) ** (1 / years) - 1
print("CAGR: ", cagr)

# Calculate max drawdown
rolling_max = data['strategy_cumulative_returns'].cummax()
daily_drawdown = data['strategy_cumulative_returns'] / rolling_max - 1.0
max_daily_drawdown = daily_drawdown.min()
print("Max Drawdown: ", max_daily_drawdown)


ImportError: cannot import name 'axes' from 'matplotlib' (C:\Users\mehdi\AppData\Local\Programs\Python\Python310\lib\site-packages\matplotlib\__init__.py)